<font color='orange' size=8><B>**LAB Assignment 3: Robot Simulation**</B></font>

<font size=5><B>Objective :</B></font>

1. ศึกษาการจำลองการเคลื่อนที่หุ่นยนต์ผ่านโปรแกรม Simulation (Gazebo)
2. ศึกษาการเชื่อมต่อ Sensor กับ ROS2
3. ศึกษาการสร้าง Algorithm สำหรับควบคุมตำแหน่งปลายมือของหุ่นยนต์


------------------------------------------------------------------------------------------------------------------------------

<center><img src="ros2_control.jpg" width="800"/></center>

นักศึกษาจะต้องออกแบบระบบควบคุมหุ่นยนต์ด้วย ROS2 โดยที่จะต้องเชื่อมต่อระหว่าง IMU กับ ROS2 ด้วย Lib 
XICRO จากนั้นจึงนำค่าที่ได้มาเป็นสัญญาณขาเข้าของ Algorithm X เพื่อควบคุมตำแหน่งปลายมือของหุ่นยนต์ใน
ระบบ Simulation มีขั้นตอนการทำและคะแนนดังต่อไปนี้ ( คะแนน 120/100 )

1. การเชื่อมต่อ IMU เข้ากับ Library XICRO (20%)
2. ทำโหนด Calibration ของ IMU (20%)     [ เป็นการคำนวณและเพิ่มค่าของ cov เข้าไปยัง message ]
3. Algorithm X สำหรับการสั่งงานหุ่นยนต์ (30%)
4. การนำไฟล์ URDF/XACRO ไปยังโปรแกรม GAZEBO (20%)
5. การติดตั้ง ROS2_CONTROLLER ไปยัง GAZEBO (20%)
6. การออกแบบโครงสร้าง Package ใน ROS2 (10%)

<font color='orange' size=6><B>ภาพรวมของหุ่นยนต์(Cocoa)ที่ได้ออกแบบมา</B></font>

<font color='orange' size=6><B>การออกแบบโครงสร้าง Package ใน ROS2</B></font>

<center><img src="img/stRos1.png" /></center>

จะมีการออกแบบโครงสร้าง Package ใน ROS2 โดยการแบ่ง Package ออกเป็น 3 ส่วน ได้แก่
1. cocoav_description : Package ที่เก็บ ไฟล์ URDF/XACRO,meshes ของหุ่นยนต์, ไฟล์ config สำหรับการตั้งค่า Controller ใน yaml รวมถึง และไฟล์ launch สำหรับการเรียกใช้งาน โดยหากเรียกใช้งานไฟล์ launch ใน package นี้ จะไม่มีการควบคุมหุ่นยนต์แต่จะเป้นแค่การแสดงหุ่นยนต์ใน GAZEBO

2. cocoav_interfaces : Package ที่ใช้ในการเก็บ msg และ srv ที่จะใช้ในการควบคุมหุ่นยนต์

3. cocoav_kinematics : Package ที่ใช้ในการเก็บโค้ดของ Algorithm X และการเชื่อมต่อ IMU (Run Xicro node แยกต่างหาก) และการคำนวณค่าต่างๆ ที่จำเป็นต่อการควบคุมหุ่นยนต์ โดยหากเรียกใช้งานไฟล์ launch ใน package นี้ จะมีการควบคุมหุ่นยนต์เพิ่มเติมจาก cocoav_description

<font color='orange' size=6><B>การนำไฟล์ URDF/XACRO ไปยังโปรแกรม GAZEBO</B></font>

<font color='orange' size=6><B>การติดตั้ง ROS2_CONTROLLER ไปยัง GAZEBO</B></font>

1. ทำการเพิ่ม Gazebo ใน launch file พร้อมกับ Node ที่ใช้ในการ spawn หุ่นยนต์ โดยจะมีการเพิ่ม joint_state_broadcaster_spawner Node ที่ใช้ในการ Publish /dynamic_joint_states, /joint_states ที่จำเป็นต่อการควบคุมหุ่นยนต์ และ การเพิ่ม robot_controller_spawner Node ที่ใช้ในการเรียกใช้งาน Controller ที่ต้องการใช้งาน

<center><img src="img/rc2ros1.png"/></center>

2. ตั้งค่า Controller Maneger ที่จะใช้ใน GAZEBO โดยการใช้ไฟล์ cocoa_controllers.yaml โดยจะเลือกใช้ Controller เป็น joint_trajectory_position_controller ซึ่งเป็น Controller ที่ใช้ในการควบคุมหุ่นยนต์ที่เป็น JointTrajectory โดยจะรับค่า JointConfiguration เพื่อใช้ในการคำนวณ Trajectory และ การควบคุมหุ่นยนต์

<center><img src="img/rc2ros2.png"/></center>

3.กำหนด command_interface และ state_interface ให้กับแต่ละ Joint

<center><img src="img/rc2ros3.png"/></center>

4. เพิ่ม Controller Maneger Plugin เข้าใน Gazebo

<center><img src="img/rc2ros4.png"/></center>

<font color='orange' size=6><B>การเชื่อมต่อ IMU เข้ากับ Library XICRO</B></font>

1. กำหนด Custom msg ที่จะใช้ในการส่งค่า IMU จาก Arudino ไปยัง ROS โดยจะมีการกำหนด 7 ค่า คือ
    - int64 time_ms : เวลา ณ ขณะที่อ่านค่า IMU (ใช้สำหรับ complementary filter ในการคำนวณค่า oreintation)(yaw ไม่สามารถประมาณให้ถูกต้องได้จึงไม่ได้ใช้ Parameter นี้)
    - float32[3] angular_velocity 
    - float32[3] linear_acceleration

<center><img src="img/imu2ros1.png"/></center>

2. ตั้งค่า Config ต่าง ๆ ใน setup_xicro.yaml โดยกำหนดชื่อ Topic เป็น /cocoav_imu_arduino

<center><img src="img/imu2ros2.png"/></center>

3.เขียนไฟล์ .ino ที่ใช้ในการส่งค่า IMU จาก Arduino ไปยัง ROS โดยจะมีการส่งค่า IMU ที่อ่านได้จาก Arduino ไปยัง ROS โดยใช้ Serial และ การส่งค่า IMU จะมีการแปลงค่าจากค่าเดิมให้เป็นค่าใหม่ โดยจะมีการแปลงค่า ดังนี้
- ไม่แปลงค่า time_ms มีค่าเป็น ms
- ไม่แปลงค่า angular_velocity โดยมีค่าเป็น deg/s
- แปลงค่า linear_acceleration เป็น m/s^2

โดยจะมีการปรับ Hz ของการ Publish ค่า IMU ให้เป็น 100 Hz (TCB0.CCMP = 2500)

<center><img src="img/imu2ros3.png"/></center>

<font color='orange' size=6><B>Algorithm X สำหรับการสั่งงานหุ่นยนต์</B></font>

1. สร้าง Node ใน ROS ที่จะใช้ในการสั่งงานหุ่นยนต์ โดยจะมีการสั่งงานหุ่นยนต์ โดยจะ
    - มีการตั้ง Delay ในการสั่งงานหุ่นยนต์ โดยจะตั้งไว้ที่ 5 วินาทีสำหรับการรอเปิด Gazebo
    - กำหนด rate ในการส่ง joint_config ให้เป็น 10 Hz และ timer
    - สร้าง Pubilsher Node สำหรับส่งค่า joint_config ไปยัง Controller ใน Gazebo
    - สร้าง Constant สำหรับการคำนวณ inverse/forward kinematics position ของหุ่นยนต์ และ Home
    - สร้าง Subscriber Node สำหรับรับค่า IMU จาก Arduino ผ่าน Calibrator Node ใน msg /cocoav_imu_calibrated

<center><img src="img/AlgX1.png"/></center>

2. กำหนดการแปลงพิกัดในการสั่งงานหุ่นยนต์ โดยจะมีการแปลงพิกัด ดังนี้

    <center><img src="img/AlgX2_1.png"/></center>

    - นำค่าตำแหน่งปัจจุบันของหุ่นยนต์มาคำนวณแปลงเป็น Cylinder Coordinate โดยจะมีการคำนวณแปลงดังนี้
        - คำนวณค่า r โดยใช้ค่ารากที่ 2 ของ x และ y ของตำแหน่งปัจจุบันของหุ่นยนต์
        - คำนวณค่า theta โดยใช้ค่า arctan2 ของ y และ x ของตำแหน่งปัจจุบันของหุ่นยนต์
        - คำนวณค่า z โดยใช้ค่า z ของตำแหน่งปัจจุบันของหุ่นยนต์


<center><img src="img/AlgX2_2.png"/></center>

3. กำหนด Algorithm สำหรับควบคุมหุ่นยนต์ดังนี้
    - รับค่าจาก IMU โดยจะใช้เพียงค่า linear_acceleration โดยจะมีการคำนวณดังนี้
        - ทำการแปลง linear_acceleration จาก Float ให้การเป็น int โดยใช้ int()
        - กำหนดค่าจะนำมาเพิ่มกับ Cylinder Coordinate ที่ 0.01 (dimension ในส่วนต่าง ๆ ของหุ่นยนต์ เช่น ความยาว link จะถูกกำหนดในหน่วย mm จึงจะใช้ค่าที่ 0.01)
    - แปลงตำแหน่งของหุ่นยนต์จาก Cartesian Coordinate ให้เป็น Cylinder Coordinate 
    - นำค่าตำแหน่งปัจจุบันของหุ่นยนต์ใน Cylinder Coordinate มาบวกหรือลบเพิ่มตาม Condition ของการควบคุมหุ่นยนต์
        - r จะมีการบวกหรือลบขึ้นกับค่า linear_acceleration ในแกน x
        - theta จะมีการบวกหรือลบขึ้นกับค่า linear_acceleration ในแกน y (เพิ่ม scale เล็กน้อยเนื่องจากหมุนช้าเกินไป)
        - z จะมีการบวกหรือลบขึ้นกับค่า linear_acceleration ในแกน z
    - แปลงตำแหน่งของหุ่นยนต์จาก Cylinder Coordinate ให้เป็น Cartesian Coordinate ดังเดิม
    - ทำการคำนวณ inverse kinematics position ของหุ่นยนต์ เพื่อหา joint_config ที่จะใช้ในการส่งให้ joint_trajectory_position_controller ซึ่งเป็น Controller ที่ใช้ในการควบคุมหุ่นยนต์ และทำการ Update ตำแหน่งของหุ่นยนต์ใหม่
        - หากไม่สามารถคำนวณ inverse kinematics ได้ จะทำการส่งค่า joint_config ที่เป็นค่าเดิมก่อนการคำนวณและไม่ทำการ Update ตำแหน่งของหุ่นยนต์

<center><img src="img/AlgX2_3.png"/></center>

3. การคำนวณ Inverse kinematic และ Constrain

<center><img src="img/AlgX3.jpg"/></center>
<center><img src="img/AlgX4.jpg"/></center>
<center><img src="img/AlgX5.jpg"/></center>
<center><img src="img/AlgX6.jpg"/></center>
<center><img src="img/AlgX7.jpg"/></center>
<center><img src="img/AlgX7_1.png"/></center>
<center><img src="img/AlgX7_2.png"/></center>

4. ทำการส่ง joint_config ให้กับ controller

<center><img src="img/AlgX8.png"/></center>

<font color='orange' size=6><B>การทำ Node Calibration ของ IMU</B></font>    

1. ในการ Calibrate IMU จะทำการสร้าง Node สำหรับการ Calibrate โดยมีการทำงานดังนี้
    - อ่านค่า IMU จาก Topic /cocoav_imu_arduino ที่ส่งมาจาก Arduino
    - ทำการเก็บค่า linear_acceleration และ angular_velocity ที่อ่านได้จาก IMU ไว้ใน data frame ของ Pandas
    - ทำการจับเวลาเริ่มต้นและจบการทำงานของ Node โดยจะกำหนดให้เก็บค่าทั้งหมด 10 วินาที
    - คำนวณ mean และ variance ของค่า linear_acceleration และ angular_velocity ที่อ่านได้จาก IMU
    - บันทึกในไฟล์ ymal

<center><img src="img/cimu1.png"/></center>
<center><img src="img/cimu2.png"/></center>

2. สร้าง Node ที่จะใช้ในการแปลงค่า linear_acceleration และ angular_velocity ตามค่าที่ Calibrate ไว้ โดยเริ่มจาก
    - โหลดไฟล์ yaml ที่เก็บค่า mean และ variance ของค่า linear_acceleration และ angular_velocity ที่อ่านได้จาก IMU
    - ลบค่า mean ออกจากค่า linear_acceleration และ angular_velocity ที่อ่านได้จาก IMU
    - แปลงค่า angular_velocity จาก deg/s เป็น rad/s (ไม่ได้แปลงมาตั้งแต่ Arduino)
    - Publish ค่า linear_acceleration และ angular_velocity ที่แปลงได้ ใน Topic /cocoav_imu_calibrated

<center><img src="img/cimu3.png"/></center>
<center><img src="img/cimu4.png"/></center>